# Jinja2 Templating and SQL Queries

Parcel out the SQL from our python modules and import them as a templatable query using jinja. Also used in terraform and dbt 

In [3]:
from jinja2 import Template
import sqlite3
from pathlib import Path

In [5]:
data_dir = Path("../data")
db_conn = sqlite3.connect(data_dir / "flights.db")

## Feeding string directly to `Template`

In [21]:
sql = """
        SELECT 
            ROW_NUMBER() OVER (ORDER BY CAST({{a_delay}} AS INTEGER) DESC) delay_rank,
            {{flight_num}},
            REPLACE(
            REPLACE(
            REPLACE({{a_port}}, ' International Airport', ''), 
            ' International', ''),
            ' Airport', '') AS {{a_port}},
            REPLACE(
            REPLACE(
            REPLACE({{d_port}}, ' International Airport', ''),
            ' International', ''),
            ' Airport', '') AS {{d_port}},
            {{a_delay}}
        FROM {{tbl_name}}
        WHERE DATE({{a_sched}}) = '{{str_date}}'
        ORDER BY delay_rank
        LIMIT {{num_delay}};
    """
sep = "__"
params = dict(
    flight_num=f"flight{sep}iata",
    a_port=f"arrival{sep}airport",
    a_delay=f"arrival{sep}delay",
    a_sched=f"arrival{sep}scheduled",
    d_port=f"departure{sep}airport",
    num_delay=3,
    str_date="2023-09-24",
    tbl_name="import_flight_records",
    not_used="not used",
)
rendered_sql = Template(sql).render(params)
print(rendered_sql)


        SELECT 
            ROW_NUMBER() OVER (ORDER BY CAST(arrival__delay AS INTEGER) DESC) delay_rank,
            flight__iata,
            REPLACE(
            REPLACE(
            REPLACE(arrival__airport, ' International Airport', ''), 
            ' International', ''),
            ' Airport', '') AS arrival__airport,
            REPLACE(
            REPLACE(
            REPLACE(departure__airport, ' International Airport', ''),
            ' International', ''),
            ' Airport', '') AS departure__airport,
            arrival__delay
        FROM import_flight_records
        WHERE DATE(arrival__scheduled) = '2023-09-24'
        ORDER BY delay_rank
        LIMIT 3;
    


In [14]:
curs = db_conn.execute(rendered_sql)
curs.fetchall()

[(1, 'MH9663', 'Los Angeles', "Chicago O'hare", '197'),
 (2, 'MH9506', 'John F Kennedy', 'Dallas/Fort Worth', '134'),
 (3, 'MH9110', "Chicago O'hare", 'Haneda', '131')]

## Importing from template

Best practice is to create an `environment` and call `env.get_template('query.sql')`

In [18]:
from jinja2 import Environment, FileSystemLoader

In [19]:
env = Environment(loader=FileSystemLoader("../sql_templates"))
template = env.get_template("delayed.sql")
rendered_temp = template.render(params)
print(rendered_temp)

SELECT 
            ROW_NUMBER() OVER (ORDER BY CAST(arrival__delay AS INTEGER) DESC) delay_rank,
            flight__iata,
            REPLACE(
            REPLACE(
            REPLACE(arrival__airport, ' International Airport', ''), 
            ' International', ''),
            ' Airport', '') AS arrival__airport,
            REPLACE(
            REPLACE(
            REPLACE(departure__airport, ' International Airport', ''),
            ' International', ''),
            ' Airport', '') AS departure__airport,
            arrival__delay
        FROM import_flight_records
        WHERE DATE(arrival__scheduled) = '2023-09-24'
        ORDER BY delay_rank
        LIMIT 3;


In [20]:
curs = db_conn.execute(rendered_temp)
curs.fetchall()

[(1, 'MH9663', 'Los Angeles', "Chicago O'hare", '197'),
 (2, 'MH9506', 'John F Kennedy', 'Dallas/Fort Worth', '134'),
 (3, 'MH9110', "Chicago O'hare", 'Haneda', '131')]

## Flow control with jinja

[for loop and if/else showcased here](https://geoffruddock.com/sql-jinja-templating/#a-usecase-for-templating-your-sql-queries)